# Capas convolucionales y de agrupación

In [2]:
pip install keras-tuner --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


## Inicialización MNIST


In [1]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner
keras.utils.set_random_seed(23); input_dim = (28, 28, 1); num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train_val = x_train_val.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train_val = np.expand_dims(x_train_val, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape)
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]


(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


## MyHyperModel
Exploramos número de filtros de la primera capa; doblamos en la segunda

In [3]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        M = keras.Sequential()
        M.add(keras.Input(shape=(28, 28, 1)))
        filters = hp.Int("filters", min_value=8, max_value=64, step=2, sampling="log")
        M.add(keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu"))
        M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        M.add(keras.layers.Conv2D(2*filters, kernel_size=(3, 3), activation="relu"))
        M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        M.add(keras.layers.Flatten())
        M.add(keras.layers.Dense(units=800, activation='relu'))
        M.add(keras.layers.Dense(10, activation='softmax'))
        opt = keras.optimizers.Adam(learning_rate=0.00168)
        M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
        return M
    def fit(self, hp, M, x, y, xy_val, **kwargs):
        factor = 0.3787; patience = 5
        reduce_cb = keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)
        early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)
        kwargs['callbacks'].extend([reduce_cb, early_cb])
        return M.fit(x, y, batch_size=256, epochs=50, validation_data=xy_val, **kwargs)

###  Experimento: exploración y resumen de resultados

In [9]:
tuner = keras_tuner.BayesianOptimization(
MyHyperModel(), objective="val_accuracy", max_trials=10, executions_per_trial=1,
overwrite=True, directory="tmp", project_name="MNIST")

In [10]:
tuner.search(x_train, y_train, (x_val, y_val))

Trial 4 Complete [00h 01m 25s]
val_accuracy: 0.9940000176429749

Best val_accuracy So Far: 0.9940000176429749
Total elapsed time: 00h 02m 58s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
64                |64                |filters

Epoch 1/50
196/196 [==============================] - 3s 12ms/step - loss: 0.1706 - accuracy: 0.9463 - val_loss: 0.0537 - val_accuracy: 0.9862 - lr: 0.0017
Epoch 2/50
196/196 [==============================] - 2s 10ms/step - loss: 0.0409 - accuracy: 0.9873 - val_loss: 0.0483 - val_accuracy: 0.9862 - lr: 0.0017
Epoch 3/50
196/196 [==============================] - 2s 11ms/step - loss: 0.0252 - accuracy: 0.9922 - val_loss: 0.0386 - val_accuracy: 0.9887 - lr: 0.0017
Epoch 4/50
196/196 [==============================] - 2s 10ms/step - loss: 0.0190 - accuracy: 0.9938 - val_loss: 0.0391 - val_accuracy: 0.9886 - lr: 0.0017
Epoch 5/50
196/196 [==============================] - 2s 11ms/step - loss: 0.0128 - accuracy: 0.9957 - val_

KeyboardInterrupt: 

In [8]:
tuner.results_summary(num_trials=4)

Results summary
Results in /tmp/MNIST
Showing 4 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
filters: 64
Score: 0.9932000041007996

Trial 00 summary
Hyperparameters:
filters: 16
Score: 0.992900013923645

Trial 01 summary
Hyperparameters:
filters: 32
Score: 0.992900013923645

Trial 03 summary
Hyperparameters:
filters: 16
Score: 0.9926999807357788


Ponemos a prueba con los **mejores valores** que os dan para cada hiperparametro

In [ ]:
num_models = 10
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
    values = best_hyperparameters[m].values
    score = best_models[m].evaluate(x_test, y_test, verbose=0)
print(f'Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}')